In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df=pd.read_csv('../input/retail-sales-forecasting/mock_kaggle.csv',index_col='data',parse_dates=True)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.plot(figsize=(12,8))

In [ ]:
df.tail

In [ ]:

test_size=21
test_ind=len(df)-test_size
test_ind

In [ ]:
train=df.iloc[:test_ind]
train

In [ ]:
test=df.iloc[test_ind:]
test

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()

In [ ]:
scaled_train=scaler.fit_transform(train)
scaled_test=scaler.transform(test)

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
length=19
generator=TimeseriesGenerator(scaled_train,scaled_train,length=length,batch_size=1)

In [ ]:
x,y=generator[0]

In [ ]:
x

In [ ]:
y

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM,Dropout

In [ ]:
scaled_train.shape

In [ ]:
n_features=scaled_train.shape[1]

In [ ]:
model=Sequential()
model.add(LSTM(100,activation='relu',input_shape=(length,n_features),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,activation='relu',input_shape=(length,n_features)))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu',input_shape=(length,n_features)))
model.add(Dense(n_features))
model.compile(optimizer='adam',loss='mse')

In [ ]:
model.summary()

In [ ]:
validation_generator=TimeseriesGenerator(scaled_test,scaled_test,length=length,batch_size=1)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
es=EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
model.fit_generator(generator,epochs=50,validation_data=validation_generator,callbacks=[es])

In [ ]:
losses=pd.DataFrame(model.history.history)
losses.plot()

In [ ]:
first_eval_batch = scaled_train[-length:]
first_eval_batch = first_eval_batch.reshape(1,length,n_features)

In [ ]:
model.predict(first_eval_batch)

In [ ]:
scaled_test[0]

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length,scaled_train.shape[1]))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
np.sqrt(mean_squared_error(test,true_predictions))

In [ ]:
full_scaler = MinMaxScaler()
scaled_full_data = full_scaler.fit_transform(df)

In [ ]:
length = 365 # Length of the output sequences (in number of timesteps)
generator = TimeseriesGenerator(scaled_full_data, scaled_full_data, length=length, batch_size=1)

In [ ]:
model=Sequential()
model.add(LSTM(100,activation='relu',input_shape=(length,n_features),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,activation='relu',input_shape=(length,n_features)))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu',input_shape=(length,n_features)))
model.add(Dense(n_features))
model.compile(optimizer='adam',loss='mse')
model.fit_generator(generator,epochs=50)

In [ ]:
forecast = []
# Replace periods with whatever forecast length you want
periods = 21

first_eval_batch = scaled_full_data[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(periods):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    forecast.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
forecast = scaler.inverse_transform(forecast)

In [ ]:
df

In [ ]:
forecast_index=pd.date_range(start='2016-07-31',periods=periods,freq='D')

In [ ]:
forecast_df = pd.DataFrame(data=forecast,index=forecast_index,
                           columns=[df.columns])

In [ ]:
forecast_df

In [ ]:
ax = df.plot()
forecast_df.plot(ax=ax,figsize=(12,9))

In [ ]:
df.plot()
forecast_df.plot()